In [ ]:
# default_exp dataset.core

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import shutil
import sys
import logging
from logging.handlers import MemoryHandler
from os.path import isdir, join, basename
from mlcore.image.pillow_tools import assign_exif_orientation, write_exif_metadata
from mlcore.io.core import create_folder, scan_files
from mlcore.tensorflow.tfrecord_builder import create_labelmap_file
from mlcore.category_tools import read_categories

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

DEFAULT_CATEGORIES_FILE = 'categories.txt'
DEFAULT_SPLIT = 0.2
TRAIN_VAL_FOLDER = 'trainval'
TRAIN_FOLDER = 'train'
VAL_FOLDER = 'val'
TEST_FOLDER = 'test'

In [ ]:
# export

logger = logging.getLogger(__name__)

# Dataset

> Dataset Notes.

Creating a dataset for a classification or segmentation task. If an annotation file is present, the annotations are also prepared.
The dataset is created based on an imageset.

## Imageset

Imagesets are collected images to build a dataset from, stored in the `imagesets` folder.
The `imagesets` folder contains the following folder structure:
- imagesets/*[imageset_type]*/*[imageset_name]*

Inside the `[imageset_name]` folder are the following files / folders
- `test/`: test images (benchmark)
- `trainval/`: training and validation images for [cross validation](https://pdc-pj.backlog.jp/wiki/RAD_RAD/Neural+Network+-+Training)
- `categories.txt`: all categories (classes) the imageset contains

## Dataset Folders

Data-sets are stored in the `datasets` base folder.
The `datasets` folder contains the following folder structure:
- datasets/*[dataset_type]*/*[dataset_name]*
where `[dataset_type]` is the same as the corresponding `[imageset_type]` and `[dataset_name]` is the same as the corresponding `[imageset_name]`.

Inside the `[dataset_name]` folder are the following files / folders
- `test/`: test set (benchmark)
- `train/`: training set
- `val/`: validation set
- `categories.txt`: all categories (classes) the dataset contains

In [ ]:
# export


class Dataset:
    def __init__(self, name, base_path, imageset_path, categories_path, dataset_type, create_tfrecord=False):
        self.name = name
        self.base_path = base_path
        self.imageset_path = imageset_path
        self.train_val_folder = join(imageset_path, TRAIN_VAL_FOLDER)
        self.test_source_folder = join(imageset_path, TEST_FOLDER)
        self.categories_path = categories_path
        self.categories = read_categories(categories_path, dataset_type)
        self.dataset_type = dataset_type
        self.create_tfrecord = create_tfrecord
        self.folder = None
        self.train_folder = None
        self.val_folder = None
        self.test_target_folder = None
        self.logger = self.get_logger()
        self.annotations = None

    def create_folders(self):
        """
        Creates the dataset folder structure, if not exist
        """

        # create data-set folder and remove previous data if exist
        self.folder = create_folder(join(self.base_path, str(self.dataset_type), self.name), clear=True)
        self.logger.info("Created folder {}".format(self.folder))
        self.train_folder = create_folder(join(self.folder, TRAIN_FOLDER))
        self.logger.info("Created folder {}".format(self.train_folder))
        self.val_folder = create_folder(join(self.folder, VAL_FOLDER))
        self.logger.info("Created folder {}".format(self.val_folder))
        if isdir(self.test_source_folder):
            self.test_target_folder = create_folder(join(self.folder, TEST_FOLDER))
            self.logger.info("Created folder {}".format(self.test_target_folder))

    def convert_annotations(self):
        """
        Converts annotations
        """
        pass

    def validate(self):
        """
        Validates, that each file has at least one annotation.
        """

        # validate only the trainval images, the test images have no annotations to validate
        # convert the annotations before doing validation
        self.convert_annotations()

        self.logger.info('Start validate imageset at {}'.format(self.train_val_folder))

        files = scan_files(self.train_val_folder)

        self.logger.info('Found {} files at {}'.format(len(files), self.train_val_folder))

        delete_annotations = {}
        used_categories = set([])

        for annotation_id, annotation in self.annotations.items():
            delete_regions = {}
            for index, region in enumerate(annotation.regions):
                len_labels = len(region.labels)
                region_valid = len_labels and len(set(region.labels) & set(self.categories)) == len_labels
                if not region_valid:
                    message = '{} : Region {} with category {} is not in category list, skip region.'
                    self.logger.info(message.format(annotation.file_name, index, ','.join(region.labels)))
                    delete_regions[index] = True
                else:
                    # update the used regions
                    used_categories.update(region.labels)

            # delete regions after iteration is finished
            for index in sorted(list(delete_regions.keys()), reverse=True):
                del annotation.regions[index]

            # validate for empty region
            if not annotation.regions:
                self.logger.info('{} : Has empty regions, skip annotation.'.format(annotation.file_path))
                delete_annotations[annotation_id] = True
            # validate for file exist
            elif annotation.file_path not in files:
                self.logger.info('{} : File of annotations do not exist, skip annotations.'.format(annotation.file_path))
                delete_annotations[annotation_id] = True
            else:
                files.pop(files.index(annotation.file_path))

        for index, file in enumerate(files):
            self.logger.info('[{}] -> {} : File has no annotations, skip file.'.format(index, file))

        # list unised categories
        empty_categories = frozenset(self.categories) - used_categories
        if empty_categories:
            self.logger.info('The following categories have no images: {}'.format(" , ".join(empty_categories)))

        # delete annotations after iteration is finished
        for index in delete_annotations.keys():
            del self.annotations[index]

        self.logger.info('Finished validate image set at {}'.format(self.train_val_folder))

    def copy(self, train_file_keys, val_file_keys, test_file_names=None):
        """
        Copy the images to the dataset.
        `train_file_keys`: The list of training image keys
        `val_file_keys`: The list of validation image keys
        `test_file_names`: The list of test image file names
        return: A tuple containing train and val annotations
        """

        self.logger.info('Start copy files from {} to {}'.format(self.imageset_path, self.folder))

        # copy the categories files
        self.logger.info('Copy file {} to {}'.format(self.categories_path, self.folder))
        shutil.copy2(self.categories_path, join(self.folder, DEFAULT_CATEGORIES_FILE))

        # if create tfrecord, create a labelmap.pbtxt file containing the categories
        if self.create_tfrecord:
            labelmap_file_name = 'label_map.pbtxt'
            labelmap_output_file = join(self.folder, labelmap_file_name)
            self.logger.info('Generate file {} to {}'.format(labelmap_file_name, self.folder))
            create_labelmap_file(labelmap_output_file, list(self.categories)[1:], 1)

        annotations_train = {}
        annotations_val = {}

        num_files = len(train_file_keys)
        self.logger.info('Start copy {} files to {}'.format(num_files, self.train_folder))

        for key in train_file_keys:
            # copy image
            annotation = self.annotations[key]
            copy_image_and_assign_orientation(self.train_val_folder, annotation.file_name, self.train_folder)

            # add annotation
            annotations_train[key] = annotation
        self.logger.info('Finished copy {} files to {}'.format(num_files, self.train_folder))

        num_files = len(val_file_keys)
        self.logger.info('Start copy {} files to {}'.format(num_files, self.val_folder))

        for key in val_file_keys:
            # copy image
            annotation = self.annotations[key]
            copy_image_and_assign_orientation(self.train_val_folder, annotation.file_name, self.val_folder)

            # add annotation
            annotations_val[key] = annotation

        self.logger.info('Finished copy {} files to {}'.format(num_files, self.val_folder))

        # copy test_files, if exist
        if test_file_names:

            num_files = len(test_file_names)
            self.logger.info('Start copy {} files to {}'.format(num_files, self.test_target_folder))

            for file_name in test_file_names:
                copy_image_and_assign_orientation(self.test_source_folder, file_name, self.test_target_folder)

            self.logger.info('Finished copy {} files to {}'.format(num_files, self.test_target_folder))

        self.logger.info('Finished copy files from {} to {}'.format(self.imageset_path, self.folder))

        return annotations_train, annotations_val

    def build(self, split=DEFAULT_SPLIT, seed=None, sample=None):
        """
        Build the data-set. This is the main logic.
        This method validates the images against the annotations,
        split the image-set into train and val on given split percentage,
        creates the data-set folders and copies the image.
        If a sample percentage is given, a sub-set is created as sample.
        `split`: The percentage of images which will be copied into the validation set
        `seed`: A random seed to reproduce splits
        `sample`: The percentage of images from train, val and test which will also from a sample set
        """

        self.logger.info('Validation set contains {}% of the images.'.format(int(split * 100)))

        # validate the image set
        self.validate()

        # sort filenames by assigned tags
        labels_to_keys = {}

        for annotation_id, annotation in self.annotations.items():
            labels = ' '.join(annotation.labels())

            if labels not in labels_to_keys:
                labels_to_keys[labels] = []
            labels_to_keys[labels].append(annotation_id)

        # split category files into train & val and create the sample split, if set
        train_file_keys = []
        val_file_keys = []
        sample_train_file_keys = []
        sample_val_file_keys = []

        for annotation_ids in labels_to_keys.values():
            train, val = split_train_val_data(annotation_ids, split, seed)
            train_file_keys.extend(train)
            val_file_keys.extend(val)

            # if a sample data set should be created, create the splits
            if sample:
                _, sample_train = split_train_val_data(train, sample, seed)
                _, sample_val = split_train_val_data(val, sample, seed)
                sample_train_file_keys.extend(sample_train)
                sample_val_file_keys.extend(sample_val)

        # scan the test images if exist
        test_file_names = list(map(lambda f: basename(f), scan_files(self.test_source_folder))) if self.test_target_folder else None
        _, sample_test_file_names = split_train_val_data(test_file_names, sample, seed) if test_file_names and sample else (None, None)

        # copy the files
        self.copy(train_file_keys, val_file_keys, test_file_names)

        if sample:
            sample_name = "{}_sample".format(self.name)

            self.logger.info('Start build {} data-set containing {}% of images at {}'.format(sample_name,
                                                                                             int(sample * 100),
                                                                                             self.base_path))

            # create the sample data-set
            sample_data_set = self.__class__(sample_name, self.base_path, self.imageset_path, self.categories_path,
                                             self.dataset_type, create_tfrecord=self.create_tfrecord)
            # assign the converted annotations
            sample_data_set.annotations = self.annotations

            # create the data set folders
            sample_data_set.create_folders()
            # copy the sample images
            sample_data_set.copy(sample_train_file_keys, sample_val_file_keys, sample_test_file_names)

            self.logger.info('Finished build {} data-set containing {}% of images at {}'.format(sample_name,
                                                                                                int(sample * 100),
                                                                                                self.base_path))

    @classmethod
    def get_logger(cls):
        """
        Configures default logging for the system.
        """
        logger = logging.getLogger(__name__)
        logger.addHandler(logging.NullHandler())
        return logger

In [ ]:
# export


def split_train_val_data(data, val_size=0.2, seed=None):
    """
    Splits the images in train and validation set
    `data`: the data to split
    `val_size`: the size of the validation set in percentage
    `seed`: A random seed to reproduce splits.
    return: the splitted train, validation images
    """

    from sklearn.model_selection import train_test_split
    train, test = train_test_split(data, random_state=seed, test_size=val_size) if len(data) > 1 else (data, [])
    return train, test

In [ ]:
# export


def copy_image_and_assign_orientation(source_folder, file_name, target_folder):
    """
    Copy an image file from source to target and assign the EXIF metadata orientation.
    `source_folder`: the source folder of the image file
    `file_name`: the image file name to copy
    `target_folder`: the target folder to copy the image file to
    return: `True` if modified EXIF metadata could be saved, else `False`
    """

    shutil.copy2(join(source_folder, file_name), target_folder)

    # rotate image by EXIF orientation metadata and remove them
    target_file = join(target_folder, file_name)
    image, exif_data, rotated = assign_exif_orientation(target_file)
    if rotated:
        write_exif_metadata(image, exif_data, target_file)

In [ ]:
# export


def input_feedback(msg, choice, choices):
    """
    User input request wrapper.
    `msg`: the message to dosplay
    `choice`: if previous choice exist
    `choices`: the possible choices
    :return: the choice input
    """

    # if decision is already made for all contents, skip feedback
    if not (choice and choice.isupper()):
        prompt = '{} \n choices: {} '.format(msg, ', '.join(['{} ({})'.format(k, v) for k, v in choices.items()]))
        while True:
            choice = input(prompt)
            if choice in choices:
                break
    return choice

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.
    """
    log_memory_handler = MemoryHandler(1, flushLevel=logging_level)
    log_memory_handler.setLevel(logging_level)

    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setLevel(logging_level)

    logger.addHandler(log_memory_handler)
    logger.addHandler(stdout_handler)

    logger.setLevel(logging_level)

    return log_memory_handler
